In [69]:
from typing import List, Callable
import random
from sys import maxsize as inf

In [89]:
def random_starts(n_cities: int, n_starts: int = 10) -> List[List[int]]:
    """
    Gera uma lista de rotas aleatórias para o problema do caixeiro viajante.
    :param n_cities: Total de cidades do problema.
    :param n_starts: Total de ortas que serão geradas
    :return: 
    """
    routes = list()
    
    for _ in range(n_starts):
        route = list()
        cities = list(range(n_cities))
        # Enquanto houverem cidades, escolhe uma aleatória como próxima  a ser visitada
        while cities: 
            city = random.choice(cities)
            cities.remove(city)
            route.append(city)
        # Faz com que volte para a primeira cidade
        route.append(route[0])
        routes.append(route)
    return routes

In [82]:
def path_cost(costs: List[List[int]], route: List[List[int]]) -> int:
    """
    Calcula o somatório do custo de cada viagem da rota.
    :param costs: Custos de cada viagem. Onde costs[i][j] é o custo de viagem da cidade j para a i.
    :param route: Caminho realizado.
    :return: O custo total da rota.
    """
    cost = 0
    for i in range(len(route)-1):
        curr_city = route[i]
        next_city = route[i+1]
        cost += costs[next_city][curr_city]
    return cost

In [111]:
def validate_childs(childs: List[List[int]]) -> List[List[int]]:
    """
    Verifica se a reprodução criou rotas inválidas,
    onde algumas cidades são visitadas mais de uma vez 
    e outras cidades não são visitadas.
    :param childs: Os 'filhos' gerados no cruzamento.
    :return: Os 'filhos' validados.
    """
    # A última cidade é ignorada, pois é a mesma que a primeira.
    x = childs[0][:-1]
    y = childs[1][:-1]
    for x_i, x_city in enumerate(x):
        if x.count(x_city) > 1:
            for y_i, y_city in enumerate(y):
                if y.count(y_city) > 1:
                    x[x_i] = y_city
                    y[y_i] = x_city
                    break
    # Volta para a primeira cidade
    x.append(x[0])
    y.append(y[0])
    return [x, y]
        

In [99]:
def reproduce(parents: List[List[int]]) -> List[List[int]]:
    """
    Divide as duas listas num ponto aleatório c e mescla as primeiras 
    partes de cada lista com as segundas partes de cada.
    :param parents: Os 'pais' que comporão os filhos.
    :return: Uma lista com 2 'filhos' compostos pelos pais.
    """
    n = len(parents[0])
    c = random.choice(range(n))
    # Mistura das listas
    childs = [parents[0][:c] + parents[1][c:], 
              parents[1][:c] + parents[0][c:]]
    # Validação para evitar que uma cidade seja visitada mais que uma vez (Exceto a primeira e última).
    
    childs = validate_childs(childs)
    return childs

In [85]:
def mutate(child: List[int], possible_mutations) -> List[int]:
    
    n = random.choice(range(len(child)))
    new_item = random.choice(possible_mutations)
    child[n] = new_item
    
    return child

In [101]:
def genetic_algorithm(population: List[List[int]], fitness: Callable, costs: List[List[int]], mutate_prob: float = 0.01, epochs: int = 10):
    
    # Custo ao longo das iterações: min, max, media
    results = [[],[],[]]
    for epoch in range(epochs):
        print(epoch)
        new_population = list()
        # Custo de cada rota
        routes_costs = [fitness(costs, route) for route in population]
        # Guarda estatísticas dos custos das rotas
        results[0].append(min(routes_costs))
        results[1].append(max(routes_costs))
        results[2].append(sum(routes_costs)/len(routes_costs))        
        # Inverso dos custos, já que o objetivo é minimizar o custo, 
        # caminhos caros terão menor probabilidade de serem escolhidos.
        inv_costs = [1/cost for cost in routes_costs]
        for _ in range(int(len(population)/2)):
            xy = random.choices(population, inv_costs, k=2)
            childs = reproduce(xy)
            # Para cada filho, tenta realizar a mutação
            for i in range(2):
                bool_mutate = random.choices([True, False], [mutate_prob, 1-mutate_prob])[0]
                if bool_mutate:
                    childs[i] = mutate(childs[i], range(len(costs)))
                # Faz com que volte para a primeira cidade
                childs[i][-1] = childs[i][0]
                
            new_population.extend(childs)
            
        population = new_population
            

In [128]:
PATH_COSTS = [
    [  0,  10,  15,  45,   5,  45,  50,  44,  30, 100,  67,  33,  90,  17,  50],
    [ 15,   0, 100,  30,  20,  25,  80,  45,  41,   5,  45,  10,  90,  10,  35],
    [ 40,  80,   0,  90,  70,  33, 100,  70,  30,  23,  80,  60,  47,  33,  25],
    [100,   8,   5,   0,   5,  40,  21,  20,  35,  14,  55,  35,  21,   5,  40],
    [ 17,  10,  33,  45,   0,  14,  50,  27,  33,  60,  17,  10,  20,  13,  71],
    [ 15,  70,  90,  20,  11,   0,  15,  35,  30,  15,  18,  35,  15,  90,  23],
    [ 25,  19,  18,  30, 100,  55,   0,  70,  55,  41,  55, 100,  18,  14,  18],
    [ 40,  15,  60,  45,  70,  33,  25,   0,  27,  60,  80,  35,  30,  41,  35],
    [ 21,  34,  17,  10,  11,  40,   8,  32,   0,  47,  76,  40,  21,  90,  21],
    [ 35, 100,   5,  18,  43,  25,  14,  30,  39,   0,  17,  35,  15,  13,  40],
    [ 38,  20,  23,  30,   5,  55,  50,  33,  70,  12,   0,  60,  30,  35,  21],
    [ 15,  14,  45,  21, 100,  10,   8,  20,  35,  43,   8,   0,  15, 100,  23],
    [ 80,  10,   5,  20,  35,   8,  90,   5,  44,  10,  80,  14,   0,  25,  80],
    [ 33,  90,  40,  18,  70,  45,  25,  23,  90,  44,  43,  70,   5,   0,  25],
    [ 25,  70,  45,  50,   5,  45,  20, 100,  25,  50,  35,  10,  90,   5,   0]
]

In [129]:
rotas = random_starts(15, n_starts=100)
genetic_algorithm(rotas, path_cost, PATH_COSTS, mutate_prob=0, epochs=100)

0
[[8, 5, 1, 10, 9, 13, 3, 11, 0, 14, 2, 12, 4, 7, 6, 8], [4, 7, 6, 8, 14, 10, 1, 13, 0, 12, 2, 3, 9, 5, 11, 4], [8, 9, 12, 11, 6, 0, 10, 3, 2, 1, 14, 4, 13, 7, 5, 8], [7, 1, 9, 10, 14, 5, 11, 3, 2, 4, 13, 12, 8, 0, 6, 7], [4, 1, 5, 2, 13, 12, 9, 0, 14, 10, 3, 11, 8, 7, 6, 4], [6, 9, 3, 11, 7, 12, 5, 8, 0, 2, 1, 14, 10, 4, 13, 6], [11, 4, 2, 14, 1, 13, 3, 12, 7, 10, 6, 9, 8, 5, 0, 11], [3, 2, 6, 12, 9, 13, 0, 8, 10, 14, 5, 11, 7, 4, 1, 3], [13, 2, 4, 7, 11, 3, 10, 14, 6, 0, 5, 1, 9, 8, 12, 13], [0, 3, 12, 13, 9, 5, 11, 6, 1, 8, 2, 4, 10, 14, 7, 0], [1, 11, 14, 7, 9, 2, 3, 12, 5, 13, 8, 0, 6, 4, 10, 1], [14, 12, 4, 0, 10, 8, 7, 3, 5, 13, 2, 11, 9, 6, 1, 14], [3, 7, 1, 2, 8, 0, 5, 14, 4, 11, 12, 10, 13, 9, 6, 3], [5, 4, 3, 9, 1, 10, 2, 14, 13, 7, 12, 11, 0, 6, 8, 5], [13, 12, 7, 4, 8, 10, 0, 3, 9, 14, 6, 2, 11, 5, 1, 13], [9, 7, 1, 13, 8, 14, 2, 12, 3, 5, 4, 6, 0, 11, 10, 9], [5, 1, 11, 0, 8, 2, 13, 3, 10, 14, 4, 12, 6, 9, 7, 5], [14, 4, 11, 6, 8, 1, 7, 2, 5, 13, 9, 10, 12, 3, 0, 14], [1

[[1], [2], [3]]

TypeError: unhashable type: 'list'

[3, 1, 1, 1, 1, 1, 1, 1, 1, 1]